In [1]:
data_flag = 'opp115'

# ohsumed: 23986
# opp115 3399
# reuters 10788

In [2]:
import pickle
import pandas as pd
import numpy as np
import utilities
import preprocess
from sklearn.model_selection import train_test_split

## parameters

In [3]:
# algorithm parameters
balance_ratio = 0.5
random_state = 1
threshold_factor = 1.5
test_size = 0.2

sim_type = 'cosine'
embedding_method = 'distiluse-base-multilingual-cased-v1' # try different embeddings and find proper one

np.random.seed(random_state)

data_paths = {'opp115'   : r'C:\Users\IsmailKaraman\workspace\data\privacy_policy_data\OPP-115_v2\majority.csv',
              'ohsumed'  : r'C:\Users\IsmailKaraman\workspace\GitHub\thesis\data\ohsumed.csv',
              'reuters'  : r'C:\Users\IsmailKaraman\workspace\GitHub\thesis\data\Reuters21578.csv'}

unlabaled_ratios = {'opp115':0.75, 'ohsumed':0.95, 'reuters':0.9}

In [4]:
import warnings
warnings.filterwarnings("ignore")

# main

In [5]:
def main(data):
    print('*'*60)
    print(data)
    # reading data
    df = utilities.read_data(data_paths[data])
    X = df['text'].apply(preprocess.preprocess_text)
    y = df.drop(['text'], axis=1)
    # ------------------------------------------------------------------------------------------------------------------------------
    # reading from a pickle instead of applying vectorization
    X_num = utilities.vectorize_data(X, embedding_method)
    X_num = pd.Series([np.squeeze(i) for i in X_num])
    X_train, X_test, y_train, y_test = train_test_split(X_num, y, test_size=test_size, random_state=random_state)
    X_labeled, X_unlabeled, y_labeled, y_unlabeled = train_test_split(X_train, y_train, test_size=unlabaled_ratios[data], 
                                                                  random_state=random_state)
    
    print(X_labeled.shape, y_labeled.shape, X_unlabeled.shape, y_unlabeled.shape)
    utilities.classifier(np.vstack(X_labeled), y_labeled, np.vstack(X_test), y_test)
    #utilities.classifier(np.vstack(X_labeled.values), y_labeled, np.vstack(X_test.values), y_test)
    # -----------------------------------------------------------------------------------------------------------------------------
    # calculation number of instances to balance dataset
    balance_ratio = 0.5
    num_of_new_instances = utilities.calculate_balancing_num_instance_multiclass(y_labeled, balance_ratio)
    # -----------------------------------------------------------------------------------------------------------------------------
    # oversampling dataset using unlabeled data with the given ratios
    validation, X_labeled, y_labeled, X_unlabeled, y_unlabeled = utilities.oversample_dataset(num_of_new_instances, 
                                                                  X_labeled, y_labeled, X_unlabeled, y_unlabeled, threshold_factor)
    # -----------------------------------------------------------------------------------------------------------------------------
    # check if the result gets better
    print(X_labeled.shape, X_unlabeled.shape, X_test.shape)
    utilities.classifier(np.vstack(X_labeled.values), y_labeled, np.vstack(X_test.values), y_test)
    
    compare_res = []
    for _, value in validation.items():
        col, _, _, y_true, y_pred = value
        compare_res.append((list(y_true.values), list(y_pred.values())))
        
    emr = 0
    accuracy = 0
    for t, p in compare_res:
        if t==p:
            emr += 1
        if t[0] == p[0]:
            accuracy +=1
        if t[1] == p[1]:
            accuracy +=1
        if t[2] == p[2]:
            accuracy +=1
    print('Metrics for the proposed algorithm ')    
    print(f'Exact match ratio : {emr/len(compare_res):.2f} ')
    print(f'Accuracy          : {accuracy/(len(compare_res)*3):.2f} ')

In [6]:
for data in data_paths.keys():
    main(data)

************************************************************
opp115
(679,) (679, 12) (2040,) (2040, 12)
LinearSVM results: 
------------------------------
hamLoss: 0.07
Exact Match Ratio: 0.46
------------------------------
Classification Report
                                      precision    recall  f1-score   support

                      Data Retention       0.11      0.23      0.15        13
                       Data Security       0.69      0.88      0.77        40
                        Do Not Track       0.86      1.00      0.92         6
          First Party Collection/Use       0.73      0.88      0.80       230
International and Specific Audiences       0.92      0.81      0.86        68
                Introductory/Generic       0.50      0.70      0.59        76
                       Policy Change       0.74      0.91      0.82        22
                Practice not covered       0.18      0.33      0.24        24
         Privacy contact information       0.54    

ValueError: Multioutput target data is not supported with label binarization

In [ ]:
stop

In [ ]:
from nltk.corpus import words
all_words = words.words()

In [ ]:
words_5len = [word.lower() for word in all_words if len(word)==5]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
data_vecs = vectorizer.fit_transform(data_words).toarray()

In [ ]:
for word in words_5len:
    if 'h' in word and 'k' in word and 'n' in word:
        print(word)

In [ ]:
from gensim.corpora.dictionary import Dictionary
common_dictionary = Dictionary(data_words)
common_corpus = [common_dictionary.doc2bow(text) for text in data_words]

In [ ]:
common_corpus

In [ ]:
data_vecs[0]

In [ ]:
data_vecs

In [ ]:
from scipy.stats import entropy
from scipy.spatial.distance import jensenshannon
from scipy.special import kl_div

In [ ]:
for word in words_5len:
    if word.startswith('se') and 'r' in word:
        print(word)

In [ ]:
letter_0 = ''
letter_1 = ''
letter_2 = 'a'
letter_3 = ''
letter_4 = ''

exist_letters = 'acs'
banned_letters = 'trdefou'

In [ ]:
def KL_divergence(p, q):
        """ Compute KL divergence of two vectors, K(p || q)."""
        return sum(p[x] * log((p[x]) / (q[x])) for x in range(len(p)) if p[x] != 0.0 or p[x] != 0)

In [ ]:
from numpy import zeros, array
from math import sqrt, log

In [ ]:
jensenshannon(p, q)

In [ ]:
filtered = [word for word in words_5len for e in exist_letters if e in word]
filtered

In [ ]:
filtered = [word for word in filtered for b in banned_letters if b in word]
filtered = [word for word in filtered if letter_0 and word[0]==letter_0]
filtered = [word for word in filtered if letter_1 and word[0]==letter_1]
filtered = [word for word in filtered if letter_2 and word[0]==letter_2]
filtered = [word for word in filtered if letter_3 and word[0]==letter_3]
filtered = [word for word in filtered if letter_4 and word[0]==letter_4]
filtered

In [ ]:
class JSD(object):
    def __init__(self):
        self.log2 = log(2)


    def KL_divergence(self, p, q):
        """ Compute KL divergence of two vectors, K(p || q)."""
        return sum(p[x] * log((p[x]) / (q[x])) for x in range(len(p)) if p[x] != 0.0 or p[x] != 0)

    def Jensen_Shannon_divergence(self, p, q):
        """ Returns the Jensen-Shannon divergence. """
        self.JSD = 0.0
        weight = 0.5
        average = zeros(len(p)) #Average
        for x in range(len(p)):
            average[x] = weight * p[x] + (1 - weight) * q[x]
            self.JSD = (weight * self.KL_divergence(array(p), average)) + ((1 - weight) * self.KL_divergence(array(q), average))
        return 1-(self.JSD/sqrt(2 * self.log2))

if __name__ == '__main__':
    J = JSD()
    p = [1.0/10, 9.0/10, 0]
    q = [0, 1.0/10, 9.0/10]
    p = data_vecs[0]
    q = data_vecs[1]
    print(J.Jensen_Shannon_divergence(p, q))

In [ ]:
if letter_2:
    print('a')

In [ ]:
for word in words_5len:
    if 'o' in word and 'u' in word and 'a' not in word and 'i' not in word and 'd' not in word:
        print(word)

In [ ]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import util

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X).toarray()